In [1]:
!nltk.download('stopwords')

import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
from pymystem3 import Mystem
from string import punctuation
import pandas as pd

stop_words = stopwords.words('russian')
stop_words.remove("не")
stop_words.remove("ни")
mystem = Mystem()
stemmer = SnowballStemmer(language='russian')
morph = pymorphy2.MorphAnalyzer()

/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [21]:
def preprocess(text, is_stem = False, is_morph = False, is_remove_stop_words = False):
    if text is None:
        text = ""
    text = text.lower()
    text = re.sub(r'\[.*?\]|\(.*?\)|\n|,|\.|\xa0|;', "", text).replace(chr(769), "")
    # tokens = word_tokenize(text, language='russian')
    if not is_remove_stop_words:
        return text
    tokens = mystem.lemmatize(text)
    if is_remove_stop_words:
        tokens = [token for token in tokens if token not in stop_words\
            and token != " " and token != '' \
            and token.strip() not in punctuation]

    if is_stem:
        tokens = list(map(lambda x: stemmer.stem(x), tokens))
    if is_morph:
        tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens))

    return ' '.join(tokens)

def preprocess_for_labeling(data):
    after_preprocess = data["original"].transform(preprocess)

    after_preprocess = after_preprocess.dropna()
    after_preprocess = after_preprocess.drop_duplicates()

    with open('/home/droman/Documents/diploma/spacy/data/data_for_labeling_with_preprocess.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(set(after_preprocess.tolist())))


def preprocess_for_classification(data, output = '/home/droman/Documents/diploma/deeppavlov_ner_3.6/data/data_with_preprocess.csv'):
    for title in ["original", "scored_text"]:
        after_preprocess = data[title].transform(preprocess)
        data[title] = after_preprocess

    data.dropna(how='any', inplace=True)
    data.drop_duplicates()

    data.to_csv(output, index=False)

In [23]:
origin_result_data = pd.read_csv("/home/droman/Documents/diploma/deeppavlov_ner_3.6/data/input/result.csv")

preprocess_for_classification(origin_result_data, '/home/droman/Documents/diploma/deeppavlov_ner_3.6/data/input/data_with_semi_preproc_with_stopwords.csv')

